In [5]:
import numpy as np
import math

In [24]:
def preprocessing(utility_matrix): 
    """
    This function applies the first preprocessing method described in paragraph 9.4.5 of the book 
    """
    M = np.copy(utility_matrix)
    
    num_rows = np.shape(M)[0]
    num_col = np.shape(M)[1]
    
    #First we subtract the mean of row i of each element m_ij 
    for i in range(num_rows):
        row_mean = np.nanmean(M[i,:]) #compute the mean for all non-NaN elements 

        for j in range(num_col):
            if not math.isnan(M[i,j]): #check if an element is NaN
                M[i,j] -= row_mean
            
    #Next we subtract the mean of column j of each element m_ji
    for j in range(num_col):
        column_mean = np.nanmean(M[:,j])

        for i in range(num_rows):
            if not math.isnan(M[j,i]):
                M[j,i] -= column_mean
    return M 

H = np.array([[5,2,4,4,3],[3,1,2,4,1],[2,np.nan,3,1,4],[2,5,4,3,5],[4,4,5,4,np.nan]]) #let the blank elements be NaN
print(H)
J = preprocessing(H)
print(J)
print(H)

[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
[[ 1.47   -1.53    0.47    0.47   -0.53  ]
 [ 1.245  -0.755   0.245   2.245  -0.755 ]
 [-0.933      nan  0.067  -1.933   1.067 ]
 [-1.7464  1.2536  0.2536 -0.7464  1.2536]
 [-0.5089 -0.5089  0.4911 -0.5089     nan]]
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]


In [25]:
def init_UV(M,d,use_mean):
    """
    This function initializes two matrices U and V for the UV-decomposition
    """
    num_rows = np.shape(M)[0]
    num_col = np.shape(M)[1]
    
    U = np.ones((num_rows,d))
    V = np.ones((d,num_col))
    
    if use_mean == True:
        scalar = np.sqrt(np.nanmean(M)/d)
        return scalar*U,scalar*V
    return U,V

In [88]:
def optimize_elements(M,U,V):
    """
    Perform a single round of optimization for the U and V matrices using the equations from page 346.
    The first two loops for U and V is for looping through all elements in the matrix,
    and the loops over j/i and k represent the sums in the expressions for x and y. 
    """
    
    num_rows = np.shape(M)[0]
    num_col = np.shape(M)[1]
    d = np.shape(U)[1]
    
    #First, we do it for U
    #there are r rows and d columns
    for r in range(num_rows):

        for s in range(d):

            #now, we apply the main formula for x from page 346: 
            total_nom = 0
            total_denom = 0 
            for j in range(num_col):
                if not math.isnan(M[r,j]):

                    eps = 0
                    for k in range(d): #for any k that is not equal to s
                        if k != s: 
                            eps += U[r][k]*V[k][j]

                    total_nom += V[s][j] * (M[r][j] - eps)

                    denom = (V[s][j])**2
                    total_denom += denom 

            U[r][s] = total_nom/total_denom

    #Next, we do it for V
    #again, there are d rows and s columns
    for r in range(d):

        for s in range(num_col):

            #now, we apply the main formula for y from page 346: 
            total_nom = 0
            total_denom = 0 
            for i in range(num_rows):
                if not math.isnan(M[i,s]):

                    eps = 0
                    for k in range(d):
                        if k != r: 
                            eps += U[i][k]*V[k][s]

                    total_nom += U[i][r] * (M[i][s] - eps)

                    denom = (U[i][r])**2
                    total_denom += denom 

            V[r][s] = total_nom/total_denom
            
    return U,V

U1,V1 = init_UV(H,2,False)
U1,V1 = optimize_elements(H,U1,V1)
P1 = np.matmul(U1,V1)
print(P1)
print(U1,V1)

[[3.50709644 3.12172258 3.98201196 3.50595269 3.90705303]
 [2.20426306 1.88236538 2.36135517 2.20797158 2.30727173]
 [2.48344164 2.14794192 2.70863877 2.48611039 2.65008201]
 [3.69321549 3.29877361 4.21353436 3.69137856 4.13559322]
 [4.11198336 3.69713843 4.73445975 4.10858678 4.64980864]]
[[2.6  1.  ]
 [1.2  1.  ]
 [1.5  1.  ]
 [2.8  1.  ]
 [3.25 1.  ]] [[0.93059527 0.88525515 1.15761199 0.92712936 1.14270093]
 [1.08754874 0.8200592  0.97222078 1.09541634 0.93603062]]


In [93]:
def converge_UV(M,U,V,max_rmse):
    """
    With this function we iterate upon the element optimization of U and V, stopping when either the RMSE
    falls below a chosen threshold or the improvement over the previous iteration becomes insignificant. 
    """
    count = 0 #count the number of iterations
    
    #initialize the errors of the previous and current step such that the condition is held for the first loop 
    rmse_old = float('inf')
    rmse_new = max_rmse + 1
    
    while rmse_new > max_rmse and (rmse_old - rmse_new > 1e-5):
        
        print('loop number {}'.format(count))
        U,V = optimize_elements(M,U,V)
        P = np.matmul(U,V)
        diff = M-P
        rmse_old = rmse_new
        rmse_new = np.sqrt(np.nanmean(diff**2)) #compute the Root-Mean Square Error between the two matrices M and P 
        count += 1
        
    return U,V,P #return the final, updated U and V

U_t,V_t = init_UV(H,2,False)
print(U_t,V_t)
test_1,test_2,test_3 = converge_UV(H,U_t,V_t,0.1)
print(H)
print(test_3)

[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]] [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
loop number 0
loop number 1
loop number 2
loop number 3
loop number 4
loop number 5
loop number 6
loop number 7
loop number 8
loop number 9
loop number 10
loop number 11
loop number 12
[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
[[4.53480161 2.22036087 3.87625217 4.55024104 2.8573573 ]
 [3.38592112 0.65468454 2.31602458 3.41721842 1.15328962]
 [1.50800046 3.8230387  3.06692852 1.45234408 3.96417101]
 [2.47548609 4.80828411 4.18875078 2.41304264 5.07359427]
 [4.08946948 4.15617136 4.73701318 4.06094609 4.681251  ]]


In [78]:
#Calculate the Square Error manually:

print(M_original)
print(U)
print(V)

print(np.count_nonzero(np.isnan(M_original)))
print(M_original.size)

SE = 0
P = np.zeros((5,5))
diff = np.zeros((5,5))

for r in range(5):
    
    for j in range(5):
        
        #print(M_original[r,j])
        
        for k in range(2):

            if k != j:
                P[r,j] = P[r,j] + U[r,k]*V[k,j] 
            else:
                P[r,j] = P[r,j] +  U[r,j]*V[k,j] 
                
        if not math.isnan(M_original[r,j]):        
            SE = SE + (M_original[r,j] - P[r,j])**2
            
            
        #P[r,j] = P[r,j] + 1
        #print(M[r,j])
        
print(SE)
RMSE = np.sqrt(SE/23)
print(RMSE)
print(P)

[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
2
25
75.0
1.805787796286538
[[2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]]
